Since long sentences have more words than short
ones, they can absorb more noise while maintaining their original class label. To compensate, we
vary the number of words changed, n, for SR, RI,
and RS based on the sentence length l with the formula n=αl, where α is a parameter that indicates
the percent of the words in a sentence are changed
(we use p=α for RD).

best dataset size is around 500 samples

The recomendation values for augmentation with around 500 samples dataset is α = 0.05 and naug = 16

In [ ]:
def get_n(text, alpha):
  l = len(text)
  n = int(alpha*l)
  return n

In [ ]:
get_n('feeling very lonely', 0.05)

0

In [ ]:
get_n('Já que amanhã a conta bancária estará recheada, que tal participar da campanha Natal Solidário do Atlântico e ajudar o Sr. Vicente e a Casa Sol Nascente?  Saiba como colaborar com a vaquinha solidária', 0.05)

10

In [ ]:
from nltk.tokenize import TweetTokenizer
tk = TweetTokenizer(reduce_len=True)
#data = []

#for x in df['Tweet']:
#      data.append((tk.tokenize(x)))
        
#data[:5]
text = 'Já que amanhã a conta bancária estará recheada, que tal participar da campanha Natal Solidário do Atlântico e ajudar o Sr. Vicente e a Casa Sol Nascente?  Saiba como colaborar com a vaquinha solidária'
teste_1 = tk.tokenize(text)
print(teste_1)  

['Já', 'que', 'amanhã', 'a', 'conta', 'bancária', 'estará', 'recheada', ',', 'que', 'tal', 'participar', 'da', 'campanha', 'Natal', 'Solidário', 'do', 'Atlântico', 'e', 'ajudar', 'o', 'Sr', '.', 'Vicente', 'e', 'a', 'Casa', 'Sol', 'Nascente', '?', 'Saiba', 'como', 'colaborar', 'com', 'a', 'vaquinha', 'solidária']


In [ ]:
len(teste_1)

37

In [ ]:
teste_2 = 'Já que amanhã a conta bancária estará recheada, que tal participar da campanha Natal Solidário do Atlântico e ajudar o Sr. Vicente e a Casa Sol Nascente?  Saiba como colaborar com a vaquinha solidária'
len(teste_2)

200

In [ ]:
get_n(teste_1, 0.05)

1

In [ ]:
get_n(teste_2, 0.05)

10

In [ ]:
#vai sem tokenizar mesmo, é o len da palavra direto

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
augmentation_type = ['Synonym Replacement', 'Random Insertion', 'Random Swap', 'Random Deletion']

In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Data Augmentation/data/en-dataset.csv', encoding="ISO-8859-1", header=None)

In [ ]:
df.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
df = df.drop(columns=[1, 2, 3, 4], axis=1)

In [ ]:
df[0] = df[0].replace([0, 2, 4], ['negative', 'neutral', 'positive'])

In [ ]:
df[0].value_counts()

positive    800000
negative    800000
Name: 0, dtype: int64

In [ ]:
positive_sentences = df.loc[df[0] == 'positive']
negative_sentences = df.loc[df[0] == 'negative']

In [ ]:
random_positive_sentences = positive_sentences.sample(n=250, random_state=42)
random_negative_sentences = negative_sentences.sample(n=250, random_state=42)

random_sentences = pd.concat([random_positive_sentences, random_negative_sentences])

In [ ]:
%%capture
!pip install clean-text
!pip install nltk

In [ ]:
import re
from cleantext import clean

def cleaner(sentence):
    sentence = re.sub("@[A-Za-z0-9]+","",sentence) 
    sentence = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", sentence) 
    sentence = " ".join(sentence.split())
    sentence = sentence.replace("#", "").replace("_", " ").replace("RT", "").replace("rt", "") 
    sentence = clean(sentence,
                  fix_unicode=True,              
                  lower=True,                     
                  no_line_breaks=False,      
                  no_punct=True)

    return sentence

random_sentences['sentences'] = random_sentences[5].map(lambda x: cleaner(x))

In [ ]:
n_values = []

for x in random_sentences['sentences']:
  n_values.append(get_n(x, 0.05))

random_sentences['max changes'] = n_values

In [ ]:
import random 

selected_type = []

for x in range(500):
  selected_type.append(random.choice(augmentation_type))

random_sentences['augmentation type'] = selected_type

In [ ]:
random_sentences.to_csv('/content/drive/MyDrive/Data Augmentation/results/500_sentences_randomizes_augmentation.csv')

In [ ]:
random_sentences

,0,5,sentences,max changes,augmentation type
212188,positive,@xnausikaax oh no! where did u order from? tha...,oh no where did u order from thats horrible,2,Random Insertion
299036,positive,A great hard training weekend is over. a coup...,a great hard training weekend is over a couple...,5,Synonym Replacement
475978,positive,"Right, off to work Only 5 hours to go until I...",right off to work only 5 hours to go until im ...,2,Synonym Replacement
588988,positive,I am craving for japanese food,i am craving for japanese food,1,Synonym Replacement
138859,positive,Jean Michel Jarre concert tomorrow gotta work...,jean michel jarre conce tomorrow gotta work 73...,3,Random Deletion
...,...,...,...,...,...
1414461,negative,@colinbaylen you guys!,you guys,0,Random Swap
1132415,negative,@joshgrieve I take it back. You called me last...,i take it back you called me last week and i s...,5,Random Deletion
1238601,negative,@binged love the Twitter handle. Is this in...,love the twitter handle is this intended to pr...,5,Random Swap
819511,negative,someone is happppy(:,someone is happppy,0,Synonym Replacement


## Full comp

In [ ]:
from time import time
from nltk.tokenize import TweetTokenizer

start_time = time()

#reduce_len nao permite a repeticao do mesmo caractere por mais de 3x | 'Helloooooooooo' => 'Hellooo'
sent = TweetTokenizer(reduce_len=True)
data = []

# Melhorar o processamento
X = random_sentences['sentences'].tolist()
Y = random_sentences[0].tolist()

# Criando uma Tuple (sentenca tokenizes, target)
for x, y in zip(X, Y):
    if y == 'negative':
        data.append((sent.tokenize(x), 1))
    else:
        data.append((sent.tokenize(x), 0))
        

print('Tempo de Processamento:', time() - start_time)
        
data[:5] 

Tempo de Processamento: 0.058281660079956055


[(['oh', 'no', 'where', 'did', 'u', 'order', 'from', 'thats', 'horrible'], 0),
 (['a',
   'great',
   'hard',
   'training',
   'weekend',
   'is',
   'over',
   'a',
   'couple',
   'days',
   'of',
   'rest',
   'and',
   'lets',
   'do',
   'it',
   'again',
   'lots',
   'of',
   'computer',
   'time',
   'to',
   'put',
   'in',
   'now'],
  0),
 (['right',
   'off',
   'to',
   'work',
   'only',
   '5',
   'hours',
   'to',
   'go',
   'until',
   'im',
   'free',
   'xd'],
  0),
 (['i', 'am', 'craving', 'for', 'japanese', 'food'], 0),
 (['jean',
   'michel',
   'jarre',
   'conce',
   'tomorrow',
   'gotta',
   'work',
   '73',
   'before',
   'though'],
  0)]

In [ ]:
%%capture
!pip install numpy==1.23.1
!pip install scipy==1.5.4
!pip install nltk 

In [ ]:
import nltk
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
import re, string
from nltk.corpus import stopwords
from cleantext import clean
nltk.download('stopwords')
STOP_WORDS = stopwords.words('english')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

def cleaned(token):
    if token == 'u':
        return 'you'
    if token == 'r':
        return 'are'
    if token == 'some1':
        return 'someone'
    if token == 'yrs':
        return 'years'
    if token == 'hrs':
        return 'hours'
    if token == 'mins':
        return 'minutes'
    if token == 'secs':
        return 'seconds'
    if token == 'pls' or token == 'plz':
        return 'please'
    if token == '2morow':
        return 'tomorrow'
    if token == '2day':
        return 'today'
    if token == '4got' or token == '4gotten':
        return 'forget'
    if token == 'dont':
        return 'do not'
    # if token == 'lol':
    #     return 'laughing out loud'
    if token == 'im':
        return 'i am'
    if token == 'n':
        return 'and'
    if token == 'amp' or token == 'quot' or token == 'lt' or token == 'gt' or token == '½25' or token == '+' or token == '-' or token == '/':
        return ''
    return token

def remove_noise(tweet_tokens):
    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        cleaned_token = cleaned(token.lower())
        
        # Tirar Tokens < 3 - Tirar pontuacoes e stop words
        if cleaned_token not in string.punctuation and len(cleaned_token) > 2 and cleaned_token not in STOP_WORDS:
            cleaned_tokens.append(cleaned_token)
            
    return cleaned_tokens

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Att Sentences

In [1]:
%%capture
!pip install --upgrade numpy
!pip install --upgrade gensim

In [4]:
!pip install --upgrade numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached numpy-1.24.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scipy 1.7.3 requires numpy<1.23.0,>=1.16.5, but you have numpy 1.24.1 which is incompatible.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.1 which is incompatible.


In [1]:
import re
import pickle
import numpy as np
import pandas as pd

import seaborn as sns
from wordcloud import WordCloud
from wordcloud import STOPWORDS
import matplotlib.pyplot as plt
import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_notebook


from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import KFold
from sklearn.metrics import roc_curve, auc
from sklearn.manifold import TSNE

from gensim.models import Word2Vec

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D, Dense, LSTM, Conv1D, Embedding

/usr/local/lib/python3.8/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Reading contractions.csv and storing it as a dict.
contractions = pd.read_csv('/content/drive/MyDrive/Data Augmentation/data/contractions.csv', index_col='Contraction')
contractions.index = contractions.index.str.lower()
contractions.Meaning = contractions.Meaning.str.lower()
contractions_dict = contractions.to_dict()['Meaning']

# Defining regex patterns.
urlPattern        = r"((http://)[^ ]*|(https://)[^ ]*|(www\.)[^ ]*)"
userPattern       = '@[^\s]+'
hashtagPattern    = '#[^\s]+'
alphaPattern      = "[^a-z0-9<>]"
sequencePattern   = r"(.)\1\1+"
seqReplacePattern = r"\1\1"

# Defining regex for emojis
smileemoji        = r"[8:=;]['`\-]?[)d]+"
sademoji          = r"[8:=;]['`\-]?\(+"
neutralemoji      = r"[8:=;]['`\-]?[\/|l*]"
lolemoji          = r"[8:=;]['`\-]?p+"

def preprocess_apply(tweet):

    tweet = tweet.lower()

    # Replace all URls with '<url>'
    tweet = re.sub(urlPattern,'<url>',tweet)
    # Replace @USERNAME to '<user>'.
    tweet = re.sub(userPattern,'<user>', tweet)
    
    # Replace 3 or more consecutive letters by 2 letter.
    tweet = re.sub(sequencePattern, seqReplacePattern, tweet)

    # Replace all emojis.
    tweet = re.sub(r'<3', '<heart>', tweet)
    tweet = re.sub(smileemoji, '<smile>', tweet)
    tweet = re.sub(sademoji, '<sadface>', tweet)
    tweet = re.sub(neutralemoji, '<neutralface>', tweet)
    tweet = re.sub(lolemoji, '<lolface>', tweet)

    for contraction, replacement in contractions_dict.items():
        tweet = tweet.replace(contraction, replacement)

    # Remove non-alphanumeric and symbols
    tweet = re.sub(alphaPattern, ' ', tweet)

    # Adding space on either side of '/' to seperate words (After replacing URLS).
    tweet = re.sub(r'/', ' / ', tweet)
    return tweet

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Data Augmentation/results/att_sentences.csv')

In [4]:
df.head()

,0,5,sentences,max changes,augmentation type,results,Unnamed: 6
0,negative,@FaketragedyCom i know,i know,0,Insertion,i know,NaN
1,negative,I.....want to.......go........home........,iwant togohome,0,Insertion,i want to go home,NaN
2,negative,@cathicks I don't get it.,i dont get it,0,Swap,i dont get it,NaN
3,negative,http://vibejournal.com/jessica_art/ needs a ne...,needs a new home,0,Swap,needs a new home,NaN
4,negative,grounded,grounded,0,Swap,grounded,NaN


In [5]:
%%time
df['processed_text'] = df['5'].apply(preprocess_apply)

CPU times: user 47.7 ms, sys: 869 µs, total: 48.6 ms
Wall time: 66 ms


In [7]:
df = df[['0', '5', 'sentences', 'max changes', 'augmentation type', 'processed_text']]

In [8]:
df

,0,5,sentences,max changes,augmentation type,processed_text
0,negative,@FaketragedyCom i know,i know,0,Insertion,<user> i know
1,negative,I.....want to.......go........home........,iwant togohome,0,Insertion,i want to go home
2,negative,@cathicks I don't get it.,i dont get it,0,Swap,<user> i do not get it
3,negative,http://vibejournal.com/jessica_art/ needs a ne...,needs a new home,0,Swap,<url> needs a new home
4,negative,grounded,grounded,0,Swap,grounded
...,...,...,...,...,...,...
495,positive,We have $89 in extra GGUI donations and I've b...,we have $89 in extra ggui donations and ive be...,6,Swap,we have 89 in extra ggui donations and i have...
496,positive,Night all. Only 5 hours of sleep last night &a...,night all only 5 hours of sleep last night i w...,6,Swap,night all only 5 hours of sleep last night a...
497,positive,"@Astro_127 Fingers crossed, and all the best t...",127 fingers crossed and all the best to you li...,6,Synonym Replacement,<user> fingers crossed and all the best to yo...
498,positive,I won't get a chance to put in my FOB question...,i wont get a chance to put in my fob question ...,6,Insertion,i will not get a chance to put in my fob quest...


In [ ]:
df.to_csv('/content/drive/MyDrive/Data Augmentation/results/att_sentences.csv')